## 12. 내장 자료형 상속과 다중 상속
- 내장 자료형 상속의 위험성
- 다중 상속과 메서드 결정 순서

### 12.1. 내장 자료형의 상속은 까다롭다

> 공식적으로 CPython은 내장 자료형의 서브클래스에서 오버라이드한 메서드가 언제 호출되는지, 혹은 호출되지 않는지에 대해 명확한 규칙을 정의하지 않는다. 일반적으로 서브클래스에서 오버라이드한 메서드는 같은 객체의 다른 내장 메서드에 의해 결코 호출되지 않는다. 예를 들어 dict의 서브 클래스에서 오버라이드한 \_\_getitem()\_\_ 메서드는 내장된 get()과 같은 메서드에 의해 호출되지 않는다.


이 절에서 설명한 문제는 C 언어로 구현된 내장 자료형의 메서드에 위임할 때만 발생하므로, 이러한 내장 자료형을 상속한 사용자 정의 클래스에만 영향을 미친다. UserDict나 MutableMapping 등 파이썬으로 구현된 클래스를 상속할 때는 이런 문제가 발생하지 않는다. 

In [2]:
class DoppelDict(dict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value]*2)

dd = DoppelDict(one=1) # __setitem__으로 생성 안됨 init에서 무시?
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3) # update 메서드에서 무시
print(dd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


내장 자료형은 '슈퍼클래스에서 구현된 메서드 안에서 호출하더라도 메서드 검색은 대상 객체의 클래스에서 시작해야 한다'는 OOP의 기본 규칙을 어기고 있다.

In [3]:
class AnswerDict(dict):
  def __getitem__(self, key):
    return 42
  
ad = AnswerDict(a='foo')
print(ad['a']) # __getitem__ 에서 키와 무관하게 42 return

d = {}
d.update(ad) # d는 기본 딕셔너리 - __getitem__ 무시?
print(d['a'])

print(d) # update 메서드는 오버라이드된 getitem 메서드 무시

# dict나 list, str 등의 내장 자료형은 사용자가 정의한 오버라이드된 메서드를 무시하므로 이 클래스들을 직접 상속하면 에러가 발생하기 쉽다.

42
foo
{'a': 'foo'}


dict 대신 collections.UserDict을 상속하면 문제 해결!

In [4]:
import collections

class DoppelDict2(collections.UserDict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value]*2)
    
dd = DoppelDict2(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3) 
print(dd)



class AnswerDict2(collections.UserDict):
  def __getitem__(self, key):
    return 42

  
ad = AnswerDict(a='foo')
print(ad['a']) 

d = {}
d.update(ad) 
print(d['a'])

print(d) 

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}
42
foo
{'a': 'foo'}


### 12.2. 다중 상속과 메서드 결정 순서

다중 상속을 지원하는 언어에서는 별개의 상위클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다. --> **다이아몬드 문제**

In [8]:
class A:
  def ping(self):
    print('ping : ', self)
    
    
class B(A):
  def pong(self):
    print('pong : ', self)
    

class C(A):
  def pong(self):
    print('PONG : ', self)
    
    
class D(B, C):
  def ping(self):
    super().ping()
    print('post-ping : ', self)
    
  def pingpong(self):
    self.ping()
    super().ping()
    self.pong()
    super().pong()
    C.pong(self)
    
    
    
d = D()
print(d.pong()) # B 메서드 출력
print(C.pong(d)) 

pong :  <__main__.D object at 0x7f0c5d7d6f28>
None
PONG :  <class '__main__.D'>
None


상속 그래프를 조회할 때 특정한 순서를 따르는데, 이 순서를  메서드 결정 순서(MRO)라고 한다. \_\_mro\_\_ 속성은 현재 클래스부터 object 클래스까지 슈퍼클래스들의 MRO를 튜플로 저장한다.

In [9]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

슈퍼클래스 메서드에 위임할 때는 내장 함수인 super()를 사용하는 것이 좋다. 하지만 MRO를 우회해서 슈퍼클래스 메서드를 직접 호출할 수 있으며, 때로는 이 방법이 더 편리하다.

프레임워크 혹은 직접 제어할 수 없는 클래스 계층 구조에 들어 있는 메서드를 호출할 때는 super()를 사용하는 것이 안전하며, 향후 프레임워크가 변경되어도 문제없이 작동한다. 

In [10]:
d = D()
d.ping()

ping :  <__main__.D object at 0x7f0c5d7d6940>
post-ping :  <__main__.D object at 0x7f0c5d7d6940>


In [11]:
d.pingpong()

ping :  <__main__.D object at 0x7f0c5d7d6940>
post-ping :  <__main__.D object at 0x7f0c5d7d6940>
ping :  <__main__.D object at 0x7f0c5d7d6940>
pong :  <__main__.D object at 0x7f0c5d7d6940>
pong :  <__main__.D object at 0x7f0c5d7d6940>
PONG :  <__main__.D object at 0x7f0c5d7d6940>


MRO는 상속 그래프를 고려할 뿐만 아니라 서브클래스 정의에 나열된 슈퍼클래스들의 순서도 고려한다.

In [13]:
print(bool.__mro__)

def print_mro(cls):
  print(', '.join(c.__name__ for c in cls.__mro__))
  
print_mro(bool)

# from frenchdeck2 import FrenchDeck2
# print_mro(FranchDeck2)
import numbers
print_mro(numbers.Integral)
import io
print_mro(io.BytesIO)
print_mro(io.TextIOWrapper)

(<class 'bool'>, <class 'int'>, <class 'object'>)
bool, int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object


### 12.3. 실세계에서의 다중 상속

### 12.4. 다중 상속 다루기

1. 인터페이스 상속과 구현 상속을 구분한다.
상속하는 이유를 명확히 하는 것이 좋다.

  - 인터페이스 상속은 is-a 관계를 의미하는 서브타입을 생성한다.
  - 구현 상속은 재사용을 통해 코드 중복을 피한다.

2. ABC를 이용해서 인터페이스를 명확히 한다.
3. 코드를 재사용하기 위해 믹스인을 사용한다.
  - 'is-a' 관계를 나타내지 않고 서로 관련 없는 여러 서브클래스에서 코드를 재사용하기 위해 설계된 클래스는 명시적으로 믹스인 클래스로 만들어야 한다. 믹스인 클래스는 새로운 자료형을 정의하지 않고 단지 재사용할 메서드들을 묶어놓을 뿐이다. 믹스인 클래스로 객체를 생성해서는 안되며, 믹스인 클래스를 상속하는 구상 클래스는 다른 클래스도 상속해야 한다.
  
4. 이름을 통해 믹스인임을 명확히 한다.
  파이썬에서는 믹스인 클래스를 명시하는 공식적인 방법이 없으므로, 클래스명 뒤에 Mixin을 붙일 것을 권장한다.
  
5. ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
  - ABC는 자료형을 정의하지만, 믹스인은 자료형을 정의하지 않는다.
  - ABC는 다른 클래스의 유일한 기저 클래스가 될 수 있지만, 믹스인은 그렇지 않다.
  - ABC에서 구현된 구상 메서드는 해당 ABC나 슈퍼클래스의 메서드만 사용할 수 있다. ABC에 정의된 구상메서드는 일종의 편의를 위한 것일 뿐이다. 

6. 두 개 이상의 구상 클래스에서 상속받지 않는다.
  구상 클래스는 0개 또는 많아야 하나의 구상 슈퍼클래스를 가져야 한다. 
  
7. 사용자에게 집합 클래스를 제공한다. 
  ABC 또는 믹스인을 조합해서 호출 코드에 유용한 기능을 제공할 수 있을 때는, 이들을 적절히 통합하는 클래스를 제공하는 것이 좋다. 
  
8. 클래스 상속보다 객체 구성을 사용하라
